# RECREANDO BASE DE DATOS PERMANENTE DE UNIDADES DE PROYECTO

In [ ]:
import pandas as pd

In [ ]:
import os

# Use absolute path based on notebook location
notebook_dir = os.path.dirname(os.path.abspath("__file__"))
file_path = os.path.join(notebook_dir, "basemaps", "UPS 14 FEB 26.xlsx")

# Check if file exists
if not os.path.exists(file_path):
	print(f"File not found: {file_path}")
	print(f"Current working directory: {os.getcwd()}")
	# Try alternative path (one level up)
	file_path = os.path.join(os.path.dirname(notebook_dir), "basemaps", "UPS 14 FEB 26.xlsx")
	
df = pd.read_excel(file_path)

## NORMALIZACIÓN nombre_up Y nombre_up_detalle

In [ ]:
# Define reserved words that should always be uppercase
reserved_words = ['I.E','I.E.', 'UTC', 'CALI', 'C.A.L.I', 'CAM', 'IPS', 'UNP', 'UP', 'SENA', 'S.A.S', 'S.A.S.', 'E.S.E', 'E.S.E.', 'C.A.L.I.', 'II']

def capitalize_with_exceptions(text):
    """Capitalize first letter of each word, keeping reserved words uppercase"""
    if pd.isna(text):
        return text
    
    words = str(text).split()
    result = []
    
    for word in words:
        # Check if word is a reserved word (case-insensitive)
        if any(word.upper() == res_word for res_word in reserved_words):
            result.append(word.upper())
        else:
            # Capitalize only first letter, rest lowercase
            result.append(word.capitalize())
    
    return ' '.join(result)

# Apply the function to both columns
df['nombre_up'] = df['nombre_up'].apply(capitalize_with_exceptions)
df['nombre_up_detalle'] = df['nombre_up_detalle'].apply(capitalize_with_exceptions)

## UNIFICACIÓN DE COORDENADAS EN UN PAR ÚNICO "coordinates"

In [ ]:
# Create a new column 'coordinates' with format "lon, lat"
df['coordinates'] = df['lon'].astype(str) + ', ' + df['lat'].astype(str)

# Drop the original lat and lon columns
df.drop(columns=['lat', 'lon'], inplace=True)

In [ ]:
df.shape

## CREACION DE UPID

In [ ]:
# Create UPIDs with special handling for Vivienda Social y Habitat
df = df.copy()

# Define vivienda_mask to identify Vivienda Social y Habitat records
vivienda_mask = (df['nombre_centro_gestor'] == 'Secretaría de Vivienda Social y Habitat') & (
    df['nombre_up'].str.contains('Vivienda', case=False, na=False)
)

# Non-vivienda: group by nombre_up and nombre_up_detalle (start at 1)
non_vivienda = df[~vivienda_mask]
group_ids = non_vivienda.groupby(['nombre_up', 'nombre_up_detalle'], dropna=False).ngroup() + 1
group_ids = group_ids.astype('Int64')

# Vivienda: unique UPID per record (start at 1)
vivienda = df[vivienda_mask]
vivienda_ids = pd.Series(range(1, len(vivienda) + 1), index=vivienda.index)

# Offset vivienda IDs to avoid collisions
offset = int(group_ids.max()) if not group_ids.empty else 0
df.loc[~vivienda_mask, 'upid'] = 'UNP-' + group_ids.astype(str)
df.loc[vivienda_mask, 'upid'] = 'UNP-' + (vivienda_ids + offset).astype('Int64').astype(str)

# Reorder columns to put upid first
cols = ['upid'] + [col for col in df.columns if col != 'upid']
df = df[cols]

In [ ]:
df

## RECONOCIMIENTO UP

In [ ]:
# Crear df_up con las columnas solicitadas
df_up = df[['upid', 'nombre_up', 'nombre_up_detalle', 'direccion', 'tipo_equipamiento', 'coordinates']].copy()

In [ ]:
df_up

### OBTENCIÓN DE "comuna_corregimiento" y "barrio_vereda"

In [ ]:
import geopandas as gpd
from shapely.geometry import Point

# Load the geojson layers (go up one level from pipelines folder to project root)
project_root = os.path.dirname(notebook_dir)
comunas_gdf = gpd.read_file(os.path.join(project_root, "basemaps", "comunas_corregimientos.geojson"))
barrios_gdf = gpd.read_file(os.path.join(project_root, "basemaps", "barrios_veredas.geojson"))

# Convert coordinates string to Point geometry
def coords_to_point(coords_str):
    """Convert 'lon, lat' string to Point geometry"""
    try:
        lon, lat = map(float, coords_str.split(', '))
        return Point(lon, lat)
    except:
        return None

# Create temporary GeoDataFrame from df_up
df_up_geo = df_up.copy()
df_up_geo['geometry'] = df_up_geo['coordinates'].apply(coords_to_point)
df_up_geo = gpd.GeoDataFrame(df_up_geo, geometry='geometry', crs='EPSG:4326')

# Spatial join with comunas_corregimientos layer
df_up_geo = gpd.sjoin(df_up_geo, comunas_gdf[['geometry', 'comuna_corregimiento']], how='left', predicate='within')
df_up['comuna_corregimiento'] = df_up_geo['comuna_corregimiento']

# Spatial join with barrios_veredas layer
df_up_geo = gpd.sjoin(df_up_geo[['nombre_up', 'nombre_up_detalle', 'direccion', 'tipo_equipamiento', 'coordinates', 'geometry']], 
                       barrios_gdf[['geometry', 'barrio_vereda']], how='left', predicate='within')
df_up['barrio_vereda'] = df_up_geo['barrio_vereda']

In [ ]:
import json

# Convert coordinates string to GeoJSON-compliant geometry
def coords_to_geojson_geometry(coords_str):
    """Convert 'lon, lat' string to RFC 7946 compliant GeoJSON Point geometry"""
    if pd.isna(coords_str):
        return None
    try:
        lon, lat = map(float, coords_str.split(', '))
        # RFC 7946 GeoJSON Point format
        return {
            "type": "Point",
            "coordinates": [lon, lat]
        }
    except:
        return None

# Replace coordinates column with geometry column
df_up['geometry'] = df_up['coordinates'].apply(coords_to_geojson_geometry)
df_up = df_up.drop(columns=['coordinates'])

# Reorder columns to put geometry at the end (common convention)
cols = [col for col in df_up.columns if col != 'geometry'] + ['geometry']
df_up = df_up[cols]

## RECONOCIMIENTO INTERVENCIONES

In [ ]:
# Crear df_int con las columnas solicitadas
requested_columns = ['upid', 'referencia_proceso', 'referencia_contrato', 'bpin', 'identificador', 'fuente_financiacion', 
                     'tipo_intervencion', 'unidad', 'cantidad', 'estado', 'presupuesto_base', 'avance_obra', 'fecha_inicio',
                     'fecha_fin', 'fecha_inauguracion', 'url_proceso', 'descripcion_intervencion', 'observaciones', 
                     'nombre_centro_gestor', 'clase_up']

# Filter only columns that exist in df
available_columns = [col for col in requested_columns if col in df.columns]
df_int = df[available_columns].copy()

In [ ]:
# Create intervencion_id with reset counter for each upid
df_int['intervencion_id'] = df_int.groupby('upid').cumcount() + 1
df_int['intervencion_id'] = df_int['upid'] + '-INT-' + df_int['intervencion_id'].astype(str)

# Reorder columns to put intervencion_id after upid
cols = ['upid', 'intervencion_id'] + [col for col in df_int.columns if col not in ['upid', 'intervencion_id']]
df_int = df_int[cols]

In [ ]:
df_int.head()

In [ ]:
# Count how many times each upid appears in df_up
upid_counts = df_up['upid'].value_counts()

# Filter only the repeated upids (appearing more than once)
repeated_upids = upid_counts[upid_counts > 1]

print(f"Total UPIDs in df_up: {len(df_up)}")
print(f"Unique UPIDs in df_up: {df_up['upid'].nunique()}")
print(f"Number of repeated UPIDs: {len(repeated_upids)}")
print(f"\nRepeated UPIDs and their counts:")
print(repeated_upids.sort_values(ascending=False))

In [ ]:
df_up

## SUBIR UP

In [ ]:
# Remove duplicate upids, keeping only the first occurrence
df_up_clean = df_up.drop_duplicates(subset=['upid'], keep='first').reset_index(drop=True)

print(f"Records before: {len(df_up)}")
print(f"Records after removing duplicates: {len(df_up_clean)}")

# Use df_up_clean for Firebase upload instead of df_up
records = df_up_clean.to_dict('records')

In [ ]:
import firebase_admin
from firebase_admin import credentials, firestore

# Initialize Firebase (only once)
if not firebase_admin._apps:
    try:
        cred = credentials.Certificate(os.path.join(project_root, "env/calitrack-secret.json"))
        firebase_admin.initialize_app(cred)
    except Exception as e:
        print(f"Error initializing Firebase: {e}")
        raise

# Get Firestore client
db = firestore.client()

# Test connection and permissions
try:
    test_doc = db.collection('unidades_proyecto').document('test').get()
    print("✓ Firebase connection successful")
except Exception as e:
    print(f"✗ Firebase permission error: {e}")
    raise

# Reference to the collection
collection_ref = db.collection('unidades_proyecto')

# Convert DataFrame to list of dictionaries
records = df_up_clean.to_dict('records')

def _insert_decimal(digits, pos):
    return digits[:pos] + "." + digits[pos:]

def _fix_coord(value, kind):
    if value is None:
        return None
    try:
        coord = float(value)
    except (TypeError, ValueError):
        return None
    max_abs = 180 if kind == "lon" else 90
    if abs(coord) <= max_abs:
        return coord
    sign = -1 if coord < 0 else 1
    digits = str(abs(int(round(coord))))
    pos = 2 if kind == "lon" else 1
    if len(digits) <= pos:
        return coord
    return sign * float(_insert_decimal(digits, pos))

def normalize_geometry(geom):
    if not isinstance(geom, dict):
        return geom
    coords = geom.get("coordinates")
    if not isinstance(coords, (list, tuple)) or len(coords) != 2:
        return geom
    lon = _fix_coord(coords[0], "lon")
    lat = _fix_coord(coords[1], "lat")
    if lon is None or lat is None:
        return geom
    return {**geom, "coordinates": [lon, lat]}

# Upload each record to Firestore
batch = db.batch()
batch_count = 0
uploaded_count = 0

for record in records:
    doc_id = record.get('upid', f'doc_{uploaded_count}')
    doc_ref = collection_ref.document(doc_id)
    clean_record = {k: v for k, v in record.items() if v is not None and (not isinstance(v, float) or not pd.isna(v))}
    if 'geometry' in clean_record:
        clean_record['geometry'] = normalize_geometry(clean_record['geometry'])
    
    batch.set(doc_ref, clean_record)
    batch_count += 1
    uploaded_count += 1
    
    if batch_count >= 500:
        try:
            batch.commit()
            print(f"Uploaded {uploaded_count} documents...")
        except Exception as e:
            print(f"Error uploading batch: {e}")
            raise
        batch = db.batch()
        batch_count = 0

# Commit any remaining documents
if batch_count > 0:
    try:
        batch.commit()
    except Exception as e:
        print(f"Error uploading final batch: {e}")
        raise

print(f"✓ Successfully uploaded {uploaded_count} documents to 'unidades_proyecto' collection")

## SUBIR INTERVENCIONES

In [ ]:
# Reference to the collection
collection_ref = db.collection('intervenciones_unidades_proyecto')

# Convert DataFrame to list of dictionaries
records = df_int.to_dict('records')

# Upload each record to Firestore
batch = db.batch()
batch_count = 0
uploaded_count = 0

for record in records:
    doc_id = record.get('intervencion_id', f'doc_{uploaded_count}')
    doc_ref = collection_ref.document(doc_id)
    clean_record = {k: v for k, v in record.items() if v is not None and (not isinstance(v, float) or not pd.isna(v))}
    
    batch.set(doc_ref, clean_record)
    batch_count += 1
    uploaded_count += 1
    if batch_count >= 500:
        try:
            batch.commit()
            print(f"Uploaded {uploaded_count} documents...")
        except Exception as e:
            print(f"Error uploading batch: {e}")
            raise
        batch = db.batch()
        batch_count = 0

# Commit any remaining documents
if batch_count > 0:
    try:
        batch.commit()
    except Exception as e:
        print(f"Error uploading final batch: {e}")
        raise

print(f"✓ Successfully uploaded {uploaded_count} documents to 'intervenciones_unidades_proyecto' collection")